## import

In [ ]:
import datetime
from utils.physics.sound_model import ISAS_grid as isg
from utils.physics.sound_model.ISAS_grid import extract_velocity_profile, compute_travel_time_optimized
import numpy as np
from cartopy.feature import LAND, COASTLINE
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib import gridspec
import xarray as xr
import os

## ISAS

In [ ]:

PATH = "/media/rsafran/CORSAIR/ISAS/86442/field/2020"
month = 1
lat_bounds= [-70,20 ]
lon_bounds = [25, 130]

# Define points A and B
lat1, lon1 = -35.5758,83.2423    # Example: Station MADE
lat2, lon2 = -50.99254334995582,35.00354027003104
resolution = 1000 # 10 km resolution

ds = isg.load_ISAS_TS(PATH, month,lat_bounds,lon_bounds, fast=False)

In [ ]:

# Compute the mean over lat and lon at each depth
sv_err_mean_profile = ds['SV_ERR'].mean(dim=["depth"], skipna=True)
sv_err_mean_profile.plot()
# # Now plot mean SV_ERR vs. depth
# fig, ax = plt.subplots(figsize=(6, 8))
#
# ax.plot(sv_err_mean_profile.values)
# ax.invert_yaxis()  # In oceanography, depth increases downward
#
# ax.set_xlabel('Mean Sound Velocity Error (m/s)')
# ax.set_ylabel('Depth (m)')
# ax.set_title('Mean Sound Velocity Error Profile (Jan-2020)')
#
# ax.grid(True)
# plt.show()


In [ ]:

# Define latitude and longitude bounds for the Indian Ocean
lat_min, lat_max = lat_bounds  # Latitude bounds (approximately from 30°S to 30°N)
lon_min, lon_max = lon_bounds  # Longitude bounds (approximately from 30°E to 120°E)

# Create a map projection (PlateCarree is a simple projection for global data)
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Add coastlines and land features
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, facecolor='lightgray')
d = 104
# Plot temperature at a specific depth (e.g., depth index 10)
temp_at_depth = ds['SOUND_VELOCITY'].isel(depth=d).squeeze()
temp_at_depth.plot(transform=ccrs.PlateCarree(), ax=ax)

# Add labels
ax.set_title(f'Mean sound Velocity in the Indian Ocean (at depth {ds["depth"][d].values} m) Jan-2020')
ax.gridlines(draw_labels=True)

plt.show()

In [ ]:


# Define points A and B
lat1, lon1 = -35.5758,83.2423    # Example: Station MADE
lat2, lon2 = -50.99254334995582,35.00354027003104
resolution = 1000         # 10 km resolution

# Generate transect
coordinates, distances, total_distance = isg._generate_coordinates_with_fixed_resolution(
    lat1, lon1, lat2, lon2, resolution
)

# Extract velocity profile along transect
velocity_profile, err, depths = isg.extract_velocity_profile(ds, coordinates,interpolate_missing=False)

# Setup figure and axes

fig = plt.figure(figsize=(18, 7), constrained_layout=True)
gs = gridspec.GridSpec(1, 2, width_ratios=[2, 1], figure=fig)

# Plot 1: velocity profile
ax1 = fig.add_subplot(gs[0])

# Plot 2: map
transform = ccrs.PlateCarree()
ax2 = fig.add_subplot(gs[1], projection=ccrs.Orthographic(central_longitude=(lon1+lon2)*0.5, central_latitude=(lat1+lat2)*0.5))

# === Plot 1: velocity Profile ===
c1 = ax1.contourf(distances, depths, velocity_profile, levels=100, cmap='viridis', extend='both')
ax1.contour(distances, depths, velocity_profile, colors='black', linewidths=0.3, levels=10)
ax1.plot(distances, depths[np.nanargmin(velocity_profile,axis=0)], color='red')
cb1 = plt.colorbar(c1, ax=ax1, label='velocity (m/s)', pad=0.02)
ax1.invert_yaxis()
ax1.set_xlabel('Distance Along Transect (m)', fontsize=12)
ax1.set_ylabel('Depth (m)', fontsize=12)
ax1.set_title('velocity Profile Along Transect', fontsize=14, weight='bold')
ax1.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)

# === Plot 2: Map with Transect ===

# Surface velocity map
temp_at_surface = ds['SOUND_VELOCITY'].sel(depth=1, method='nearest').squeeze()
c2 = ax2.contourf(
    ds['longitude'], ds['latitude'], temp_at_surface,
    levels=10, transform=transform
)
cb2 = plt.colorbar(c2, ax=ax2, orientation='vertical', label='Surface velocity (m/s)', pad=0.03)

# Map features
ax2.add_feature(LAND, edgecolor='black', color='lightgray', zorder=1)
ax2.add_feature(COASTLINE, linewidth=0.7)

# Transect line and endpoints
transect_lats, transect_lons = zip(*coordinates)
ax2.plot(transect_lons, transect_lats, color='blue', linewidth=1,
         transform=transform, label='Transect')

ax2.plot(lon1, lat1, 'go', transform=transform, label='Point A')
ax2.text(lon1 + 1, lat1 + 1, 'A', transform=transform, fontsize=10, weight='bold')

ax2.plot(lon2, lat2, 'ro', transform=transform, label='Point B')
ax2.text(lon2 + 1, lat2 + 1, 'B', transform=transform, fontsize=10, weight='bold')

# Set map extent with margin
margin = 10
ax2.set_extent([
    min(lon1, lon2) - margin, max(lon1, lon2) + margin,
    min(lat1, lat2) - margin, max(lat1, lat2) + margin
], crs=transform)

# Gridlines
gl = ax2.gridlines(draw_labels=True, linewidth=0.5, color='gray', linestyle='--', alpha=0.7)
gl.right_labels = False
gl.top_labels = False

# Title and legend
ax2.set_title('Transect Location on Map', fontsize=14, weight='bold')
ax2.legend(loc='best', fontsize=10)

plt.show()



In [ ]:
svmin = ds['SOUND_VELOCITY'].min(dim="depth", skipna=True)
svdept = ds['SOUND_VELOCITY'].interp(depth=1105,  method="nearest")
svmax = ds["SOUND_VELOCITY"].max(dim="depth", skipna=True)
svmean = ds["SOUND_VELOCITY"].median(dim="depth", skipna=True)
svvar= ds["SOUND_VELOCITY"].var(dim="depth", skipna=True)

In [ ]:
(svmin).plot()

In [ ]:
ds["SOUND_VELOCITY"].idxmin(dim="depth", skipna=True).plot()

### resampeling

In [ ]:


def resample(ds, how='min', depth=None):
    sv = ds["SOUND_VELOCITY"]

    if how == 'min':
        sv_sel = sv.min(dim="depth", skipna=True)
    elif how == 'fix_depth':
        if depth is None:
            raise ValueError("You must provide a 'depth' when how='fix_depth'")
        sv_sel = sv.interp(depth=depth, method="nearest")
    elif how == "median":
        sv_sel = sv.median(dim="depth", skipna=True)
    else:
        raise ValueError(f"Unknown method: {how}")

    # Masque des profondeurs où la vitesse = sélectionnée
    mask = sv == sv_sel

    # Incertitude correspondante
    sv_err = ds["SV_ERR"].where(mask).max(dim="depth", skipna=True)

    # Profondeur correspondante (utile)
    depth_sel = ds["depth"].where(mask).max(dim="depth", skipna=True)

    result = xr.Dataset({
        "SOUND_VELOCITY": sv_sel,
        "SV_ERR": sv_err,
        "DEPTH": depth_sel
    })
    return result



In [ ]:
result = resample(ds)
result["SOUND_VELOCITY"].plot()
plt.plot( 77.28737580952215,-38.50144448850686, 'r+')
plt.show()

In [ ]:


PATH = "/media/rsafran/CORSAIR/ISAS/86442/field/2020"
lat_bounds = [-70, 20]
lon_bounds = [25, 130]
LAT_BOUNDS = [-60, 5]
LON_BOUNDS = [35, 120]
method = 'min'
year = '2018'
PATH = f"/media/rsafran/CORSAIR/ISAS/86442/field/{year}"
out_dir = f"/media/rsafran/CORSAIR/ISAS/extracted/{year}/"
os.makedirs(out_dir, exist_ok=True)
# for month in range(1, 13):
#     print(f"Traitement du mois {month}...")
#     ds = isg.load_ISAS_TS(PATH, month, lat_bounds, lon_bounds, fast=False)
#
#     # Appliquer ton resample
#     ds_resampled = resample(ds, how=method)
#     outfile = f"sound_velocity_{method}_{month}.nc"
#     ds_resampled.to_netcdf(out_dir + outfile, mode="w", format="NETCDF4")
#
#     print(f"Sauvegarde effectuée dans {out_dir + outfile}")

In [ ]:
# def load_ISAS_extracted(ISAS_Repertory, month,):
#     arr = os.listdir(ISAS_Repertory)
#     file_list = [os.path.join(ISAS_Repertory, fname) for fname in arr if fname.endswith('.nc')]
#     print(f"ISAS_Repertory {ISAS_Repertory}")
#     ds = xr.open_dataset(file_list[month - 1], engine='netcdf4', decode_times=False)
#     return ds

result = isg.load_ISAS_extracted(out_dir, 4)

In [ ]:
lats, lons = np.array([c[0] for c in coordinates]), np.array([c[1] for c in coordinates])
# Distances entre points avec pyproj (vectorisé, rapide)
segment_lengths = distances

In [ ]:
result.latitude

In [ ]:
from pyproj import Geod
import numpy as np
geod = Geod(ellps="WGS84")

lat0, lon0 = -41.23, 62.96
lats = result.latitude.values
lons = result.longitude.values

lon2d, lat2d = np.meshgrid(lons, lats)

_, _, dist_m = geod.inv(lon0 * np.ones_like(lon2d),
                        lat0 * np.ones_like(lat2d),
                        lon2d, lat2d)


In [ ]:
# Construire un DataArray à partir de dist_m
da_dist = xr.DataArray(
    dist_m,
    dims=("latitude", "longitude"),
    coords={"latitude": lats, "longitude": lons},
    name="DISTANCE"
)

# Ajouter à ton Dataset
result = result.assign(DISTANCE=da_dist)

result["TRAVEL_TIME"] = result["DISTANCE"] / result["SOUND_VELOCITY"]

In [ ]:
from joblib import Parallel, delayed

def compute_cell(i, j):
    return isg.compute_travel_time(lat0, lon0, lat2d[i,j], lon2d[i,j], result, verbose=False)[0]

vel_flat = Parallel(n_jobs=-1)(
    delayed(compute_cell)(i,j)
    for i in range(dist_m.shape[0])
    for j in range(dist_m.shape[1])
)

vel = np.array(vel_flat).reshape(dist_m.shape)


In [ ]:
# Define start and end points
lat1, lon1 = -31.5758,83.2423    # Example: Station MADE
lat2, lon2 = -59.99254334995582,35.00354027003104  # Example: Station NEAMS
depth = 1200    # Depth in meters

# Compute travel time
for i in range(0,25) :
    travel_time, err, total_distance = isg.compute_travel_time(lat1, lon1, lat2, lon2, result,resolution=10, interpolate_missing=True)

# Total distance: 2148.56 km
# Segments: 716
# Velocity: mean=1487.07 m/s, min=1486.78, max=1487.55
# Travel time: 1444.83 s (0.40 hr)
# Travel time between A and B at depth 1500 m: 1444.83 seconds
# Total distance: 2148.56 km

# Print results
print(f"Travel time between A and B at depth {depth} m: {travel_time:.2f} seconds")
print(f"Total distance: {total_distance / 1000:.2f} km")

In [ ]:
import os
from utils.physics.sound_model.spherical_sound_model import GridEllipsoidalSoundModel
import utils.physics.sound_model.ISAS_grid as isg
import datetime
PATH = "/media/rsafran/CORSAIR/ISAS/86442/field/2020"
lat_bounds = [-70, 20]
lon_bounds = [25, 130]
LAT_BOUNDS = [-60, 5]
LON_BOUNDS = [35, 120]
method = 'min'
year = '2018'
PATH = f"/media/rsafran/CORSAIR/ISAS/86442/field/{year}"
out_dir = f"/media/rsafran/CORSAIR/ISAS/extracted/{year}/"
os.makedirs(out_dir, exist_ok=True)
result = isg.load_ISAS_extracted(out_dir, 4)

In [ ]:
# Define start and end points
lat1, lon1 = -31.5758,83.2423    # Example: Station MADE
lat2, lon2 = -59.99254334995582,35.00354027003104  # Example: Station NEAMS
depth = 1200    # Depth in meters

travel_time, err, total_distance = isg.compute_travel_time(lat1, lon1, lat2, lon2, result,resolution=10, interpolate_missing=True)
print(travel_time)
ISAS_PATH = "/media/rsafran/CORSAIR/ISAS/extracted/2018"
arr = os.listdir(ISAS_PATH)
file_list = [os.path.join(ISAS_PATH, fname) for fname in arr if fname.endswith('.nc')]
SOUND_MODEL = GridEllipsoidalSoundModel(file_list)
SOUND_MODEL.get_sound_travel_time((lat1, lon1), (lat2, lon2),date = datetime.datetime(2018,4,1) )

In [ ]:
for i in range(500):
    travel_time = SOUND_MODEL.get_sound_travel_time((lat1+i/70, lon1),(lat2, lon2),date = datetime.datetime(2018,4,1))

In [ ]:
for i in range(500):
    travel_time, err, total_distance = isg.compute_travel_time(lat1, lon1, lat2, lon2, result,resolution=10, interpolate_missing=True)

In [ ]:
lat_res = result.latitude.diff("latitude").mean().item()
lon_res = result.longitude.diff("longitude").mean().item()
print(lat_res, lon_res)

In [ ]:
def compute_cell_optimized(i, j):
    return SOUND_MODEL.get_sound_travel_time((lat0, lon0),( lat2d[i,j], lon2d[i,j]),date = datetime.datetime(2018,4,1))

vel_flat = Parallel(n_jobs=-1)(
    delayed(compute_cell_optimized)(i,j)
    for i in range(dist_m.shape[0])
    for j in range(dist_m.shape[1])
)

vel_op = np.array(vel_flat).reshape(dist_m.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter( lon2d, lat2d, c= (vel_op))#, cmap="seismic" ,vmax = 0.1, vmin =-0.1)
plt.colorbar()

In [ ]:
diff =vel_op - result["DISTANCE"]/1485
plt.scatter( lon2d, lat2d, c= diff, cmap="seismic" , vmax=50, vmin =-50)
plt.colorbar()

In [ ]:
from pyproj import Geod
geod = Geod(ellps="WGS84")  # use WGS84 ellipsoid

az12, az21, total_distance = geod.inv(lon1, lat1, lon2, lat2)

# Compute number of points
num_points = int(np.floor(total_distance / 10)) + 1
distances = np.linspace(0, total_distance, num_points)
print(distances)

In [ ]:
import netCDF4 as nc
import os
ISAS_PATH = "/media/rsafran/CORSAIR/ISAS/extracted/2018"
arr = os.listdir(ISAS_PATH)
file_list = [os.path.join(ISAS_PATH, fname) for fname in arr if fname.endswith('.nc')]
data = nc.Dataset(file_list[0])
data

In [ ]:
def load_NetCDF(NetCDF_path, data_name, lat_bounds, lon_bounds, lat_name="lat", lon_name="lon"):
    """ Function to load a NetCDF file.
    :param NetCDF_path: Path of the data, in NetCDF format.
    :param data_name: Name of the main variable in the netCDF, like "t_an" in WOA temperature.
    :param lat_bounds: Latitude bounds as a size 2 float array (°) (points outside these bounds will be ignored).
    :param lon_bounds: Longitude bounds as a size 2 float array (°) (points outside these bounds will be ignored).
    :param lat_name: The name used to represent the available latitudes in the NetCDF.
    :param lon_name: The name used to represent the available longitudes in the NetCDF.
    :return: The data, lat, lon, lat resolution and lon resolution together with the initial NetCDF object.
    """

    data = nc.Dataset(NetCDF_path)

    lat_resolution = data.variables[lat_name][1] - data.variables[lat_name][0]
    lon_resolution = data.variables[lon_name][1] - data.variables[lon_name][0]

    # index bounds in the lat/lon grid given the lat/lon bounds and the resolution
    lat_bounds = (max(0,int((lat_bounds[0] - data.variables[lat_name][0]) / lat_resolution)),
                  max(0,int((lat_bounds[1] - data.variables[lat_name][0]) / lat_resolution)))
    lon_bounds = (max(0,int((lon_bounds[0] - data.variables[lon_name][0]) / lon_resolution)),
                  max(0,int((lon_bounds[1] - data.variables[lon_name][0]) / lon_resolution)))

    lat, lon = (np.array(data.variables[lat_name])[lat_bounds[0]:lat_bounds[1]+1],
                np.array(data.variables[lon_name])[lon_bounds[0]:lon_bounds[1]+1])

    # load the right variable, keeping nan values as nan in the numpy array
    grid = np.array(data.variables[data_name][lat_bounds[0]:lat_bounds[1], lon_bounds[0]:lon_bounds[1]]
                    .filled(fill_value=np.nan))
    if len(grid.shape) == 3:  # particular case of unneeded temporal axis
        grid = grid[0]

    return grid, lat, lon, data

In [ ]:
lat_bounds= [-70,20 ]
lon_bounds = [25, 130]

load_NetCDF(file_list[0],'SOUND_VELOCITY',lat_bounds,lon_bounds, lat_name="latitude", lon_name="longitude")

In [ ]:
import numpy as np
import xarray as xr
from pyproj import Geod

geod = Geod(ellps="WGS84")

def geodetic_distance(lon, lat, lon0, lat0):
    _, _, dist = geod.inv(
        lon0 * np.ones_like(lon),
        lat0 * np.ones_like(lat),
        lon, lat
    )
    return dist

def harmonic_mean_weighted(ds, var="SOUND_VELOCITY", lat0=None, lon0=None):
    """
    Calcule la moyenne harmonique de SOUND_VELOCITY dans ds
    par rapport au point (lat0, lon0).
    """
    lats = ds.latitude.values
    lons = ds.longitude.values
    lon2d, lat2d = np.meshgrid(lons, lats)

    # Distances depuis (lat0, lon0)
    dist = geodetic_distance(lon2d, lat2d, lon0, lat0)
    dist = xr.DataArray(dist, coords={"latitude": lats, "longitude": lons}, dims=("latitude", "longitude"))

    # Poids = 1/distance (éviter 0)
    weights = 1 / (dist + 1e-9)

    v = ds[var]

    # Moyenne harmonique pondérée
    num = weights.sum()
    den = (weights / v).sum()

    v_harm = num / den
    return v_harm


In [ ]:
import numpy as np
import xarray as xr
from scipy.stats import hmean
from pyproj import Geod

# Initialisation de pyproj
geod = Geod(ellps="WGS84")

# Coordonnées de référence
lat0, lon0 = -41.23, 62.96

# Extraction des données
lats = result.latitude.values
lons = result.longitude.values
values = result["SOUND_VELOCITY"].values

# Création d'un meshgrid pour les coordonnées
lon2d, lat2d = np.meshgrid(lons, lats)

# Calcul des distances géodésiques
_, _, dist = geod.inv(lon0 * np.ones_like(lon2d),
                      lat0 * np.ones_like(lat2d),
                      lon2d, lat2d)

# Calcul des poids (1 / distance)
weights = 1.0 / (dist + 1e-9)  # Ajout d'un petit epsilon pour éviter la division par zéro

# Calcul de la moyenne harmonique pondérée
hm_weighted = hmean(values, weights=weights)

# Création d'un DataArray xarray pour la moyenne harmonique
hm_field = xr.DataArray(
    hm_weighted,
    coords={"latitude": lats, "longitude": lons},
    dims=("latitude", "longitude")
)

# Affichage du résultat
hm_field.plot()


In [ ]:
import numpy as np
import xarray as xr
from pyproj import Geod
from scipy.interpolate import RegularGridInterpolator

geod = Geod(ellps="WGS84")


def geodesic_points(lat0, lon0, lat1, lon1, n_points=100):
    """
    Retourne n_points le long de la géodésique entre (lat0, lon0) et (lat1, lon1)
    """
    points = geod.npts(lon0, lat0, lon1, lat1, n_points)
    # points est une liste de tuples (lon, lat)
    lons = [lon0] + [p[0] for p in points] + [lon1]
    lats = [lat0] + [p[1] for p in points] + [lat1]
    return np.array(lats), np.array(lons)

def harmonic_along_line_xarray(da, lat0, lon0, lat1, lon1, n_points=100):
    # Points le long de la géodésique
    lats_line, lons_line = geodesic_points(lat0, lon0, lat1, lon1, n_points)

    # Interpolation nearest
    sampled = da.interp(latitude=xr.DataArray(lats_line, dims="points"),
                        longitude=xr.DataArray(lons_line, dims="points"),
                        method="nearest")


    # Calcul distances géodésiques entre les points pour les poids
    _, _, dists = geod.inv(lons_line[:-1], lats_line[:-1], lons_line[1:], lats_line[1:])
    dists = np.insert(dists, 0, 1e-9)  # éviter div par zéro
    weights = 1.0 / dists

    # Moyenne harmonique pondérée
    hm = weights.sum() / ((weights / sampled["SOUND_VELOCITY"].values).sum())
    return float(hm)

def harmonic_field_geodesic(ds, var="SOUND_VELOCITY", lat0=None, lon0=None, n_points=100):
    """
    Calcule le champ 2D des moyennes harmoniques le long de la géodésique entre (lat0, lon0) et chaque point de la grille
    """
    # Extraire valeurs 2D
    if "time" in ds.dims:
        values = ds[var].isel(time=0).values
    else:
        values = ds[var].values

    lat_grid = ds.latitude.values
    lon_grid = ds.longitude.values

    result_field = np.zeros_like(values, dtype=float)

    # Boucler sur chaque cellule
    for i, lat1 in enumerate(lat_grid):
        for j, lon1 in enumerate(lon_grid):
            result_field[i, j] = harmonic_along_line_xarray(ds, lat0, lon0, lat1, lon1, n_points=100)
    return xr.DataArray(result_field, coords={"latitude": lat_grid, "longitude": lon_grid},
                        dims=("latitude", "longitude"))


## SoundVelocityGrid

In [ ]:
from utils.physics.sound_model.ellipsoidal_sound_model import GridEllipsoidalSoundModel as GridSoundModel
from datetime import datetime
from utils.physics.geodesic.distance import distance_point_point
import os
import matplotlib.pyplot as plt
import numpy as np
# uncomment the wanted sound model
ISAS_PATH = "/media/rsafran/CORSAIR/ISAS/extracted/2018"
arr = os.listdir(ISAS_PATH)
file_list = [os.path.join(ISAS_PATH, fname) for fname in arr if fname.endswith('.nc')]
sound_model = GridSoundModel(file_list, loader='ISAS')
# sound_model = HomogeneousSoundModel(sound_speed=1480)

lat1, lon1 = -31.5758,83.2423    # Example: Station MADE
lat2, lon2 = -59.99254334995582,35.0035 # Example: Station NEAMS
pos_1 = (lat1, lon1)
pos_2 = (lat2, lon2)


propagation_time = sound_model.get_sound_travel_time(pos_1, pos_2, datetime.now())
print(f"Propagation time between {pos_1} and {pos_2}: {propagation_time:.2f}s")
print(f"(Distance is {distance_point_point(pos_1, pos_2)/1_000:.2f} km)")

In [ ]:
plt.imshow(sound_model.models[0].sv_err, origin="lower")

In [ ]:
sound_model.models[0].data.shape

In [ ]:
import netCDF4 as nc
data = nc.Dataset(file_list[0])

In [ ]:
sound_model.models[0].get_sound_speed_with_uncertainty(pos_1, pos_2)

In [ ]:
lats = sound_model.models[0].lat
lons = sound_model.models[0].lon
# pos_1 = (-38.50144448850686, 77.28737580952215)
hmeans = []
errs = []
for lat in lats:
    for lon in lons:
        pos_2 = (lat, lon)
        hmean, err = sound_model.models[0].get_sound_speed_with_uncertainty(pos_1, pos_2)
        hmeans.append(hmean)
        errs.append(err)

In [ ]:
lat_bounds= [-70,20 ]
lon_bounds = [25, 130]

lat_min, lat_max = lat_bounds  # Latitude bounds (approximately from 30°S to 30°N)
lon_min, lon_max = lon_bounds  # Longitude bounds (approximately from 30°E to 120°E)


# Create a map projection (PlateCarree is a simple projection for global data)
fig, axs = plt.subplots(1,2,figsize=(10, 6))#, subplot_kw={'projection': ccrs.PlateCarree()})
# for ax in axs:
#     ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
#     # Add coastlines and land features
#     ax.coastlines()
#     ax.add_feature(cfeature.BORDERS, linestyle=':')
#     ax.add_feature(cfeature.LAND, facecolor='lightgray')

x,y = np.meshgrid(lons, lats)
hmeans = np.reshape(hmeans, (len(lats), len(lons)))
errs = np.reshape(errs, (len(lats), len(lons)))



s0 = axs[0].scatter(x,y, c=hmeans )
s= axs[1].scatter(x,y, c=errs)
plt.colorbar(s)
plt.colorbar(s0)

In [ ]:
plt.hist(errs.flatten(), bins=100)

In [ ]:
sound_model.models[0].get_along_path(pos_1, pos_2)[0]

In [ ]:
plt.hist(hmeans.flatten(), bins=100)


In [ ]:
import scipy
import numpy as np

def monte_carlo_test(c, sigma_c, n_samples=100000):
    # Tirages aléatoires des c_i
    c_samples = np.random.normal(loc=c, scale=sigma_c, size=(n_samples, len(c)))
    c_eq_samples = scipy.stats.hmean(c_samples, axis=1)

    return np.nanmean(c_eq_samples), np.nanstd(c_eq_samples), c_eq_samples

# Exemple
c = sound_model.models[0].get_along_path(pos_1, pos_2)[0]# vitesses
sigma_c = sound_model.models[0].get_along_path(pos_1, pos_2, which='sv_err')[0] # incertitudes absolues
ds = sound_model.models[0].get_along_path(pos_1, pos_2)[1]
print("ds: ", ds)
mc_mean, mc_std, samples = monte_carlo_test(c, sigma_c)

print("Monte Carlo mean:", mc_mean)
print("Monte Carlo std :", mc_std)
mc_mean

In [ ]:
def harmonic_mean_with_uncertainty(c, sigma_c, ds):
    """
    Harmonic mean of sound speed with propagated uncertainty.

    :param c: array of sound speeds [m/s]
    :param sigma_c: array of uncertainties (absolute, same units as c)
    :param ds: step length along path [m]
    :return: (c_eq, sigma_c_eq)
    """
    L = len(c) * ds
    T = np.sum(ds / c)  # total "time"
    c_eq = L / T

    # propagation of uncertainty
    coeffs = (ds / c**2) * sigma_c
    sigma_c_eq = (L / T**2) * np.sqrt(np.sum(coeffs**2))

    return c_eq, sigma_c_eq
harmonic_mean_with_uncertainty(c, sigma_c, ds)

In [ ]:
import geopy
from pyproj import Geod

def get_geodesic(p1, p2, step):
    """Sample points along the geoid separating p1 and p2."""
    d = geopy.distance.geodesic(p1, p2).nautical / 60
    npoints = max(int(np.ceil(d / step)), 1)
    geoid = Geod(ellps="WGS84")
    points = np.array(geoid.npts(p1[1], p1[0], p2[1], p2[0], npoints))  # (lon, lat)
    real_step = d / npoints
    return points[:, ::-1], real_step

def get_geodesic(p1,p2, step):
    from pyproj import Geod
    geod = Geod(ellps="WGS84")  # use WGS84 ellipsoid
    _, _, d = geod.inv(p1[1], p1[0], p2[1], p2[0])
    npoints = max(int(np.floor(d / (step*111_000 ))), 2)
    coordinates = geod.npts(p1[1], p1[0], p2[1], p2[0], npoints, initial_idx=1, terminus_idx=1) # (lon, lat)
    real_step_m = d / npoints #m
    real_step = real_step_m / 111_000
    coordinates = np.reshape(coordinates, (npoints,2))
    return coordinates[:, ::-1], real_step, real_step_m

pts, step,n = get_geodesic(pos_1, pos_2, 0.5)

In [ ]:
pos_1, pos_2

In [ ]:
lat_bounds= [-70,20 ]
lon_bounds = [25, 130]

lat_min, lat_max = lat_bounds  # Latitude bounds (approximately from 30°S to 30°N)
lon_min, lon_max = lon_bounds  # Longitude bounds (approximately from 30°E to 120°E)


# Create a map projection (PlateCarree is a simple projection for global data)
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Add coastlines and land features
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, facecolor='lightgray')

plt.plot(pts[:,1], pts[:,0])
plt.scatter(x,y,c=hmeans)